In [22]:
import pandas as pd

In [23]:
data_path = '/home/ec2-user/SageMaker/Effective-Policies/Data'

In [24]:
def read_table(rct):
    reg_table = pd.read_csv('{}/regression_table.csv'.format(data_path))
    

In [25]:
def get_pareto_reward(year, policy_id, state):
    '''
    When Pareto decides a policy for a given state in a given year,
        This function will randomly select a valid experiment for the state
        And return its reward function
    '''    
    rct = get_rct(year, policy_id, state)
    
    reward = read_table(rct)
    
    return reward

In [26]:
!pwd

/home/ec2-user/SageMaker/effective-policies-2/Regressions


In [102]:
def get_rct(year, policy_id, state):

    mapper = pd.read_csv('{}/Policy_ID_RCT_Table.csv'.format(data_path))

    subset = mapper[ mapper['PolicyID'] == policy_id]
    
    try:

        if state in set(subset['Treatment State']):
            s2 = subset[ subset['Treatment State'] == state]['Control State'].as_matrix()[0]
            rct =  [state, s2]

        else:

            # find a similar state that DID enact that policy

            # check if this one is a control state, if so, use it
            if state in set(subset['Control State']):
                s2 = subset[ subset['Control State'] == state]['Treatment State'].as_matrix()[0]
                rct = [state, s2]

            # otherwise, search for similar treatment states 
            else:

                potential_states = list(set(subset['Treatment State']))
                reg_table = pd.read_csv('{}/regression_table.csv'.format(data_path))

                similar_states =  reg_table[ (reg_table['State 1'] == state) | (reg_table['State 2'] == state)]

                candidates = similar_states[ (similar_states['State 1'].isin(potential_states)) | (similar_states['State 2'].isin(potential_states))]      

                rct_selection = candidates.loc[ abs(candidates['Similarity T Value']).idxmin() ]

                rct = candidates.loc[ abs(candidates['Similarity T Value']).idxmin() ][ ['State 1', 'State 2']].tolist()    

    except:
        print ("Sorry! We're not supporting that policy or state at the moment. ")
        return None
                
                
    return rct

rct = get_rct(2014, 30, 'Texas')
print (rct)

Sorry! We're not supporting that policy or state at the moment. 
None
